In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from collections import OrderedDict

import matplotlib.pyplot as plt
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [2]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        #self.w = torch.nn.Parameter(torch.rand([1]))
        #self.b = torch.nn.Parameter(torch.rand([1]))


        self.net =nn.Sequential(nn.Conv2d(1,64,kernel_size=3),
                                nn.ReLU(),
                                nn.MaxPool2d((2,2), stride=2),
                                nn.Conv2d(64, 128, kernel_size=3),
                                nn.ReLU(),
                                nn.MaxPool2d((2,2), stride=2),
                                nn.Conv2d(128, 64, kernel_size=3),
                                nn.ReLU(),
                                nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(nn.Linear(64, 20, bias=True),
                                nn.ReLU(),
                                nn.Linear(20, 10, bias=True))

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size,-1))

In [5]:
mnist_testset = datasets.FashionMNIST(root='./data', train=False,
download=True, transform = ToTensor())

100%|███████████████████████████| 26421880/26421880 [00:27<00:00, 961787.53it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|█████████████████████████████████| 29515/29515 [00:00<00:00, 182300.89it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|████████████████████████████| 4422102/4422102 [00:01<00:00, 2672369.23it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████████████████████████████| 5148/5148 [00:00<00:00, 1385008.15it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [7]:
batch_size = 4


In [9]:
test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
model = CNNClassifier()
model = torch.load("./ModelFiles/model.pt")
model.to(device)

CNNClassifier(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)

In [12]:
print("Model's state_dict:")
for param_tensor in model.state_dict().keys():
    print(param_tensor, "\t",model.state_dict()[param_tensor].size())
print()

Model's state_dict:
net.0.weight 	 torch.Size([64, 1, 3, 3])
net.0.bias 	 torch.Size([64])
net.3.weight 	 torch.Size([128, 64, 3, 3])
net.3.bias 	 torch.Size([128])
net.6.weight 	 torch.Size([64, 128, 3, 3])
net.6.bias 	 torch.Size([64])
classification_head.0.weight 	 torch.Size([20, 64])
classification_head.0.bias 	 torch.Size([20])
classification_head.2.weight 	 torch.Size([10, 20])
classification_head.2.bias 	 torch.Size([10])



In [13]:
model.eval()
correct = 0
total = 0
for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = model(tinputs)
#Select the predicted class label which has the
# highest value in the output layer
    _, predicted = torch.max(toutputs, 1)
    print("True label:{}".format(tlabels))
    print('Predicted: {}'.format(predicted))
# Total number of labels
    total += tlabels.size(0)
# Total correct predictions
    correct += (predicted == tlabels).sum()
accuracy = 100.0 * correct / total
print("The overall accuracy is {}".format(accuracy))

/opt/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400440653/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


True label:tensor([9, 2, 1, 1], device='cuda:0')
Predicted: tensor([6, 8, 0, 2], device='cuda:0')
True label:tensor([6, 1, 4, 6], device='cuda:0')
Predicted: tensor([8, 0, 0, 8], device='cuda:0')
True label:tensor([5, 7, 4, 5], device='cuda:0')
Predicted: tensor([2, 4, 8, 8], device='cuda:0')
True label:tensor([7, 3, 4, 1], device='cuda:0')
Predicted: tensor([0, 6, 0, 8], device='cuda:0')
True label:tensor([2, 4, 8, 0], device='cuda:0')
Predicted: tensor([8, 8, 0, 8], device='cuda:0')
True label:tensor([2, 5, 7, 9], device='cuda:0')
Predicted: tensor([8, 2, 4, 0], device='cuda:0')
True label:tensor([1, 4, 6, 0], device='cuda:0')
Predicted: tensor([8, 6, 0, 8], device='cuda:0')
True label:tensor([9, 3, 8, 8], device='cuda:0')
Predicted: tensor([6, 8, 6, 2], device='cuda:0')
True label:tensor([3, 3, 8, 0], device='cuda:0')
Predicted: tensor([8, 8, 6, 8], device='cuda:0')
True label:tensor([7, 5, 7, 9], device='cuda:0')
Predicted: tensor([4, 2, 4, 3], device='cuda:0')
True label:tensor([6